In [3]:
import folium
import requests
from openrouteservice import client
from shapely import geometry
from shapely.geometry import Point, LineString, Polygon, MultiPolygon
from hackaton_directions import *

In [4]:
# Set up the connection to the openrouteservice api
api_key = '5b3ce3597851110001cf6248bef01ef5bac84751865f2ee2c21dcf5c'  # Individual api key
ors = client.Client(key=api_key)  # Create client with api key

# get data with the road works from Rostok 
url = 'https://geo.sv.rostock.de/download/opendata/baustellen/baustellen.json'
rostock_json = requests.get(url).json()  # Get data as JSON

# chek the json has been correctly parsed
rostock_json

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'uuid': '068bea0d-aca7-4e07-9d9b-b59c46d97f28',
    'kreis_name': None,
    'kreis_schluessel': None,
    'gemeindeverband_name': None,
    'gemeindeverband_schluessel': None,
    'gemeinde_name': None,
    'gemeinde_schluessel': None,
    'strasse_name': None,
    'strasse_schluessel': None,
    'sparte': 'Straßenbau',
    'von': 'Poppendorf',
    'nach': 'Clearum',
    'baubeginn': '2023/09/18 00:00:00+02',
    'bauende': '2023/12/15 00:00:00+01',
    'verkehrsbeeintraechtigungen': 'zeitweise halbseitige Sperrung',
    'baumassnahme': 'Arbeiten an Geh-/Radwegen'},
   'geometry': {'type': 'Point', 'coordinates': [12.2974457, 54.1291892]}},
  {'type': 'Feature',
   'properties': {'uuid': '08ff1b24-c07d-41de-bd72-868847a9869e',
    'kreis_name': 'Rostock',
    'kreis_schluessel': '13003',
    'gemeindeverband_name': 'Rostock, Hanse- und Universitätsstadt',
    'gemeindeverband_schluessel': '130030000',
   

In [5]:
# get the coordinates of the road interruptions from the Rostok JSON
site_coords=[]
for site_data in rostock_json['features']:
    site_coords.append(site_data['geometry']['coordinates'])

# some plotting on a map
map_params = {'tiles': 'OpenStreetMap',
              'location': ([54.13207, 12.101612]),
              'zoom_start': 12}
map1 = folium.Map(**map_params) 
sites_poly = []
for coord in site_coords:
    folium.features.Marker(list(reversed(coord)),
                        popup='Construction point<br>{0}'.format(coord)).add_to(map1)


map1

In [6]:
# Get the driving directions by car given a start point and end point

startpoint = [12.066989, 54.104956]
endpoint = [12.153884, 54.096297]
route, summary = driving_directions(ors,startpoint,endpoint)

# first output is a list of all the coordinates to identify the road to be taken on the map, second
# output gives a summary of the route (distance in meters and duration in seconds)
summary

{'distance': 8409.3, 'duration': 1350.1}

In [7]:
# just some plots of the route on a map

# set the centerpoint on the route as the new midpoint of the map (pass y before x)
map_params.update({'location': ([(startpoint[1]+endpoint[1])/2,
                                 (startpoint[0]+endpoint[0])/2] ),
                   'zoom_start': 13})
map2 = folium.Map(**map_params)

# add the route to the map. Again, folium need to flip x and y in coordinates.
folium.PolyLine(locations=[(y, x) for x, y in route], 
                name='Route without construction sites',
                color='#FF0000',
                weight = 4,
                overlay=True).add_to(map2)

map2

In [8]:
# pass the list of all the road works in addition to the start and end point to the method
# that provides directions avoiding the roadworks

route, summary = driving_directions_avoiding_points(ors,startpoint,endpoint,site_coords)

# compare the new summary with the original
summary

{'distance': 10292.1, 'duration': 1661.6}

In [9]:
# add the new route to the map.
folium.PolyLine(locations=[(y, x) for x, y in route], 
                name='Route avoiding construction sites',
                color='#00FF00',
                weight = 4,
                overlay=True).add_to(map2)

map2
